In [1]:
import sys
import os

sys.path.insert(0, 'compiled_protobufs')


import json
from draw import get_taksgraph_visualization
from google.protobuf.json_format import MessageToDict
import json
from google.protobuf.json_format import Parse
from pyserini.search import LuceneSearcher

from taskmap_pb2 import TaskMap

/home/ubuntu/task-search-quality/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-10 13:00:42.300320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 13:00:43.460054: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-10 13:00:43.460157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_

In [3]:
import time

# from models_indexes.bm25_model import BM25Model

# model = BM25Model(domain="COOKING")

# model.search("chicken soup with homemade broth", 10)

def get_taskgraph(id, domain):
    # print(os.path.join(os.getcwd(), "indexes", domain, "system_index_sparse"))
    taskmap_index_path = os.path.join(os.getcwd(), "indexes", domain, "system_index_sparse")
    searcher = LuceneSearcher(index_dir=taskmap_index_path)
    id = id.replace('\n','')
    doc = searcher.doc(docid=id)
    taskmap_json = json.loads(doc.raw())['recipe_document_json']
    taskmap = Parse(json.dumps(taskmap_json), TaskMap())
    return taskmap
    

# domains = ["diy"]
domains = ["cooking"]
k = 10
for domain in domains:
    empty_annotations = f"measurements/{domain}/empty_annotations/empty_annotations-{k}.csv"
    qrels = f"measurements/judgments/{domain}.qrels"
    with open(qrels, "r") as f_qrles:
        qrles = [q for q in f_qrles]
        done_annoations_count = len(qrles)
        print(f"Number of annotations for {domain}:", len(qrles))
    with open(empty_annotations, "r") as f_empty:
        annotations = [annotation for annotation in f_empty]
        annotaitons_count = len(annotations)

    annotations_tracker = done_annoations_count
    for annotation in annotations[done_annoations_count+1:]:

        annnotation = annotations[annotations_tracker+1]
        query_id, target_query, doc_id = annnotation.split(",")
        doc_id = doc_id.replace('\n','')
        taskmap = get_taskgraph(doc_id, domain)
        
        print(f"Annotation for {domain} number {annotations_tracker}/{annotaitons_count}", flush = True)
        print("Query: ", target_query , flush = True)
        print(taskmap.title, taskmap.source_url, flush = True)
        time.sleep(0.05)

        rating = int(input("Relevance rating:"))
        print("Rating:", rating)
        with open(qrels, "a") as f_qrles:
            # print(query_id, target_query, id)
            f_qrles.write(f"{query_id} Q0 {doc_id} {rating}\n")
        annotations_tracker += 1
        

        


Number of annotations for cooking: 1440
Annotation for cooking number 1440/4424
Query:  how to make a traditional greek dinner
Sharp Cheddar "Lasagna" Made Over http://www.kraftrecipes.com/recipes/sharp-cheddar-lasagna-made-over-115991.aspx
Rating: 0
Annotation for cooking number 1441/4424
Query:  how to make a traditional greek dinner
Greek Chicken and Rice Dinner http://www.kraftrecipes.com/recipes/greek-chicken-rice-dinner-63328.aspx
Rating: 0
Annotation for cooking number 1442/4424
Query:  how to make a traditional greek dinner
Greek-Style Salmon Salad http://www.kraftrecipes.com/recipes/greek-style-salmon-salad-126813.aspx
Rating: 1
Annotation for cooking number 1443/4424
Query:  how to make a traditional greek dinner
Grilled Chicken, Red Onion, and Mint Kebabs with Greek Salad http://www.epicurious.com/recipes/food/views/grilled-chicken-red-onion-and-mint-kebabs-with-greek-salad-105308
Rating: 0
Annotation for cooking number 1444/4424
Query:  how to make a traditional greek dinne

ValueError: invalid literal for int() with base 10: ''